# Lab | Making predictions with logistic regression

In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

In [11]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine

import getpass  # To get the password without showing the input

In [12]:
password = getpass.getpass()

········


# Instructions

# 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).

In [39]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query_X = """
SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.language_id,
    f.length,
    f.rating,
    f.special_features,
    r.rental_date,
    r.customer_id
FROM
    film AS f
LEFT JOIN
    inventory AS i ON f.film_id = i.film_id
LEFT JOIN
    rental AS r ON i.inventory_id = r.inventory_id
WHERE
    r.rental_date BETWEEN '2005-05-01' AND '2005-05-31'
"""

data_X = pd.read_sql_query(query_X, engine)
data_X.head()

,film_id,title,release_year,language_id,length,rating,special_features,rental_date,customer_id
0,80,BLANKET BEVERLY,2006,1,148,G,Trailers,2005-05-24 22:53:30,130
1,333,FREAKY POCUS,2006,1,126,R,"Trailers,Behind the Scenes",2005-05-24 22:54:33,459
2,373,GRADUATE LORD,2006,1,156,G,"Trailers,Behind the Scenes",2005-05-24 23:03:39,408
3,535,LOVE SUICIDES,2006,1,181,R,"Trailers,Behind the Scenes",2005-05-24 23:04:41,333
4,450,IDOLS SNATCHERS,2006,1,84,NC-17,Trailers,2005-05-24 23:05:21,222


In [40]:
data_X.shape

(993, 9)

# 2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.

In [41]:
query_y = """
SELECT
    f.film_id,
    f.title,
    MAX(CASE WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 1 ELSE 0 END) AS rented_in_may
FROM
    film AS f
LEFT JOIN
    inventory AS i ON f.film_id = i.film_id
LEFT JOIN
    rental AS r ON i.inventory_id = r.inventory_id
GROUP BY
    f.film_id, f.title
"""

data_y = pd.read_sql_query(query_y, engine)
data_y.head()

,film_id,title,rented_in_may
0,1,ACADEMY DINOSAUR,1
1,2,ACE GOLDFINGER,0
2,3,ADAPTATION HOLES,0
3,4,AFFAIR PREJUDICE,1
4,5,AFRICAN EGG,1


# 3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

In [31]:
X = pd.read_sql_query(query_X, connection_string)
y = pd.read_sql_query(query_y, connection_string)

In [32]:
X

,film_id,title,release_year,language_id,length,rating,special_features,rental_date,customer_id
0,80,BLANKET BEVERLY,2006,1,148,G,Trailers,2005-05-24 22:53:30,130
1,333,FREAKY POCUS,2006,1,126,R,"Trailers,Behind the Scenes",2005-05-24 22:54:33,459
2,373,GRADUATE LORD,2006,1,156,G,"Trailers,Behind the Scenes",2005-05-24 23:03:39,408
3,535,LOVE SUICIDES,2006,1,181,R,"Trailers,Behind the Scenes",2005-05-24 23:04:41,333
4,450,IDOLS SNATCHERS,2006,1,84,NC-17,Trailers,2005-05-24 23:05:21,222
...,...,...,...,...,...,...,...,...,...
988,907,TRANSLATION SUMMER,2006,1,168,PG-13,Trailers,2005-05-30 23:25:14,311
989,627,NORTH TEQUILA,2006,1,67,NC-17,"Commentaries,Deleted Scenes,Behind the Scenes",2005-05-30 23:29:22,593
990,782,SHAKESPEARE SADDLE,2006,1,60,PG-13,"Commentaries,Behind the Scenes",2005-05-30 23:47:56,123
991,451,IGBY MAKER,2006,1,160,NC-17,"Trailers,Behind the Scenes",2005-05-30 23:54:19,513


In [33]:
y

,film_id,title,rented_in_may
0,1,ACADEMY DINOSAUR,1
1,2,ACE GOLDFINGER,0
2,3,ADAPTATION HOLES,0
3,4,AFFAIR PREJUDICE,1
4,5,AFRICAN EGG,1
...,...,...,...
995,996,YOUNG LANGUAGE,0
996,997,YOUTH KICK,0
997,998,ZHIVAGO CORE,1
998,999,ZOOLANDER FICTION,1


# 4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [44]:
X.dtypes

film_id                      int64
title                       object
release_year                 int64
language_id                  int64
length                       int64
rating                      object
special_features            object
rental_date         datetime64[ns]
customer_id                  int64
dtype: object

In [45]:
X.isna().sum()

film_id             0
title               0
release_year        0
language_id         0
length              0
rating              0
special_features    0
rental_date         0
customer_id         0
dtype: int64

In [49]:
# There are two categorical columns in the dataframe 'X'

X['rating'].value_counts(dropna=False)

PG-13    234
NC-17    201
R        197
PG       181
G        180
Name: rating, dtype: int64

In [46]:
X['special_features'].value_counts(dropna=False)

Trailers,Commentaries,Behind the Scenes                   85
Commentaries,Deleted Scenes,Behind the Scenes             80
Deleted Scenes,Behind the Scenes                          77
Commentaries                                              75
Deleted Scenes                                            74
Trailers                                                  68
Trailers,Commentaries,Deleted Scenes,Behind the Scenes    67
Behind the Scenes                                         66
Trailers,Commentaries                                     65
Trailers,Behind the Scenes                                61
Commentaries,Behind the Scenes                            59
Trailers,Commentaries,Deleted Scenes                      56
Commentaries,Deleted Scenes                               56
Trailers,Deleted Scenes                                   56
Trailers,Deleted Scenes,Behind the Scenes                 48
Name: special_features, dtype: int64

# 5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

In [ ]:
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)

# 6. Evaluate the results.

In [ ]:
# Make predictions on the test set
y_pred = log_reg_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)